In [2]:
# DLCD itteration comparison tool 
# specific vegitation change detection? 

year_in = '2001'

name_file= '_Trees_to_wetland_'

classes_of_int='34_11'

class_values_a = [34] #class number for old DLCD class

class_values = [11] #class number for changed DLCD class



#load requiered modules 
import numpy as np
import rasterio
#from matplotlib import pyplot as plt #don't need unless want to plot
import xarray as xr
import pandas as pd
import matplotlib as mpl
from matplotlib import pyplot as plt

from scipy import ndimage
from scipy.ndimage import measurements
from scipy import stats

import os

#set up filepath due to folder re-arrangement 
os.chdir('/g/data/r78/ck9738')

#define colour map for plotting 
cmap = mpl.colors.ListedColormap(['xkcd:black',
'xkcd:grey',
'xkcd:black',
'xkcd:medium blue',
'xkcd:robin egg blue',
'xkcd:plum',
'xkcd:pink purple',
'xkcd:raspberry',
'xkcd:old pink',
'xkcd:pale pink',
'xkcd:warm pink',
'xkcd:tealish',
'xkcd:black',
'xkcd:black',
'xkcd:pumpkin orange',
'xkcd:white',
'xkcd:yellow',
'xkcd:black',
'xkcd:apricot',
'xkcd:eggshell',
'xkcd:black',
'xkcd:black',
'xkcd:black',
'xkcd:black',
'xkcd:umber',
'xkcd:tan',
'xkcd:black',
'xkcd:black',
'xkcd:black',
'xkcd:black',
'xkcd:black',
'xkcd:deep green', 
'xkcd:medium green',
'xkcd:light sage',
'xkcd:slime green',
'xkcd:light grey'])

#load in geotifs using rasterio, trun into an xarray :
#enter filename as a string, include subfolders!!
#previous_version DLCD found in old_DLCD, new in new_DLCD.

#use this command to print a list of all file in respective folder : 

def load_gfif(file_name, x):
    #file_string = 'old_DLCD/'+file_name+'.tif'

    dataset = rasterio.open(file_name) #open using Raterio
    {i: dtype for i, dtype in zip(dataset.indexes, dataset.dtypes)}
    data_array = dataset.read(x) #read selected band into np array
    #year_xr = xr.DataArray(data_array) #turn into Xr array for fuctionality #Now don't need xr yet
    return(data_array) #return np array into program 
    
second_year = int(year_in)
second_year = second_year + 1
second_year = str(second_year)

old_dlcd_string='old_DLCD/DLCDv2_'+year_in+'_'+second_year+'1.tif'
new_dlcd_band= int(year_in[1:4])

old_version = load_gfif(old_dlcd_string,1)#for old dlcd files all only have ONE band (are single year)

new_version = load_gfif('new_DLCD/DLCDv2_2016_new.tif',new_dlcd_band)#for new DLCD band number corrosponds to year in following table

# define function to return the number of pixles which are class A in old and class B in new:
#fuction returns an array, int: array with 1= occurance of class overlap and 0= everything else.

def count_overlap_array(old_np,A,new_np,B):
    raw_np = np.ones_like(old_np) #generate a np with the same size as the dlcd but full of 1 values
    raw_xr = xr.DataArray(raw_np) #turn to xarray for it's useful fuctionality
    count_old = raw_xr.where(old_np==A) # make an xr where 1= presance of class A, 0= absence 
    count_new = count_old.where(new_np==B) # make an xr where 1= overlap of classes A&B
    count_class = count_new.fillna(0) #replaces all the nan with 0
    howmuch_name = np.count_nonzero(count_class) #counts the number of non 0 in the xaray


    return(count_class) #returns an array


this_change = count_overlap_array(old_version,class_values_a,new_version,class_values)
if this_change.sum() > 0:
    labled_array, number_of_objects = measurements.label(this_change)
    name_point_list = np.unique(labled_array)
    list_of_points=measurements.center_of_mass(this_change,labled_array,name_point_list)
    output_list = [(b,a) for a,b in list_of_points]
                
    plot_name = classes_of_int+name_file+year_in
    fig, ax = plt.subplots(figsize=(40,40))
    plt.title(plot_name)
    plt.scatter(*zip(*output_list),c='r', s=10)
    ax.imshow(new_version, cmap)
    plt.savefig('output_images/'+plot_name+'.png')
    #plt.show()
    plt.close()




/g/data/v10/public/modules/agdc-py3-env/20171214/envs/agdc/lib/python3.6/site-packages/scipy/ndimage/measurements.py:1301: RuntimeWarning: invalid value encountered in true_divide
  for dir in range(input.ndim)]
